# 새로운 txt 파일을 넣어서 사용하는 카테고리 라벨링 자동화 프로그램

In [31]:

import pandas as pd
import numpy as np
import pickle
from joblib import load

data=[]
# 텍스트 파일 줄바꿈 기준으로 읽기
with open('txt/쿠팡정리2023.txt', 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()  # 줄바꿈 기준으로 나눔
    for line in lines:
        data.append(line.split('\t'))  # 각 줄을 탭 기준으로 나눔

# 데이터를 DataFrame으로 변환 (예: 각 줄을 한 열로)
df = pd.DataFrame(data, columns=['주문일','주문ID','사업자코드','사업자명','상품ID','#','상품명','카테고리','가격'])

# 필요한 열만 추출출
columns_to_drop = df.columns[[1,2,3,4,5,7]]
df = df.drop(columns=columns_to_drop)

# 빈 문자열을 NaN으로 변환
df.replace("", np.nan, inplace=True)

# 비어있는 행 표시시
print("비어있는 행")
print(df.isnull().sum())
print()

# NaN이 포함된 행 삭제
df.dropna(how='all', inplace=True)

print(df.head())  # 결과 확인
print()
print(df.isnull().sum())

비어있는 행
주문일    0
상품명    0
가격     0
dtype: int64

                   주문일                          상품명      가격
0  2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900
1  2023-12-21 08:09:34                     쿠크다스 케이크  13,020
2  2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500
3  2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300
4  2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040

주문일    0
상품명    0
가격     0
dtype: int64


## modeling.ipynb에서 추출한 모델과 벡터라이저 라벨인코더 사용

In [32]:
# 모델 로드
with open('model/logistic_regression_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# 벡터라이저 로드
with open('model/tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)
    
#라벨인코더 로드드       
label_mapping = load('model/label_mapping.joblib')

print("모델과 벡터라이저가 성공적으로 로드되었습니다.")

모델과 벡터라이저가 성공적으로 로드되었습니다.


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV



# 새로운 데이터에 대한 예측
df_tfidf = vectorizer.transform(df['상품명'])  # 기존 벡터라이저 사용
df['카테고리'] = loaded_model.predict(df_tfidf)


#라벨 0,1 -> 한글변환 함수수
df['카테고리'] = df['카테고리'].map(label_mapping)

# 결과 확인
print("\n예측된 결과:")
print(df)


예측된 결과:
                     주문일                          상품명      가격       카테고리
0    2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900    음식 및 음료
1    2023-12-21 08:09:34                     쿠크다스 케이크  13,020    음식 및 음료
2    2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500    음식 및 음료
3    2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300    음식 및 음료
4    2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040    음식 및 음료
..                   ...                          ...     ...        ...
436  2023-01-09 01:10:56        카누 다크로스트 아메리카노 원두커피믹스  16,340    음식 및 음료
437  2023-01-08 03:56:31                고르고 바른 마스크 대형   7,900  장비 및 생활용품
438  2023-01-05 02:42:00                   공차 허니자몽블랙티   7,380    음식 및 음료
439  2023-01-05 02:42:00           ipTIME USB허브 UC304  14,240  장비 및 생활용품
440  2023-01-05 02:42:00        알티스트 핫초코 한잔에 당분이 단 1g   8,900    음식 및 음료

[441 rows x 4 columns]


In [ ]:
# 라벨링된 파일 저장
# 저장 후 버전 업그레이드 수정하기기
df.to_excel('excel/coupang_V1.0', index=False)